In [ ]:
import numpy as np #alloys numpy arrays to be used
import matplotlib as mpl #see next comment
import matplotlib.pyplot as plt #allows use of plt. to plot figures
import pandas as pd
import networkx as nx #https://networkx.readthedocs.io/en/stable/index.html  #Aric A. Hagberg, Daniel A. Schult and Pieter J. Swart, “Exploring network structure, dynamics, and function using NetworkX”, in Proceedings of the 7th Python in Science Conference (SciPy2008), Gäel Varoquaux, Travis Vaught, and Jarrod Millman (Eds), (Pasadena, CA USA), pp. 11–15, Aug 2008
import os
%matplotlib inline

In [ ]:
Header = ["scale", "id", "desc_scale", "desc_id", "num_prog", "pid", "upid", "desc_pid", "phantom", "sam_mvir", "mvir", "rvir", "rs", "vrms", "mmp", "scale_of_last_MM", "vmax", "x", "y", "z", "vx", "vy", "vz", "Jx", "Jy", "Jz", "Spin", "Breadth_first_ID", "Depth_first_ID", "Tree_root_ID", "Orig_halo_ID", "Snap_num", "Next_coprogenitor_depthfirst_ID", "Last_progenitor_depthfirst_ID", "Last_mainleaf_depthfirst_ID", "Tidal_Force", "Tidal_ID", "Rs_Klypin", "Mvir_all", "M200b", "M200c", "M500c", "M2500c", "Xoff", "Voff", "Spin_Bullock", "b_to_a", "c_to_a", "Ax", "Ay", "Az", "b_to_a500c", "c_to_a500c", "Ax500c", "Ay500c", "Az500c", "T/|U|", "M_pe_Behroozi", "M_pe_Diemer" ]

Finds the first merger tree above a given size

In [ ]:
Size = 12
Files = os.listdir("/data/B-P50Mpc/Processed")
for File in Files:
    df = pd.read_csv("/data/B-P50Mpc/Processed/%s" %(File), delim_whitespace = True, names = Header)
    if(np.log10(df[df.scale == 1.00231].M200c[0])>Size):
        print(File)
        break

In [ ]:
df = pd.read_csv("/data/B-P50Mpc/Processed/Tree2812608373.dat", delim_whitespace = True, names = Header)

In [ ]:
df.M200c = np.log10(df.M200c) #turns m200c into a log10 m200c

In [ ]:
print(df.loc[:,['id','scale','M200c', 'upid', 'mmp', 'desc_id']]) #prints the things we care about from the DF

In [ ]:
grouped = df.groupby('scale') # creates and prints a grouped version of the dataframe
for name, group in grouped:
    print(group.loc[:,['id','scale','M200c', 'upid', 'mmp', 'desc_id']])

Function to correctly postion nodes in higherarchical tree

https://stackoverflow.com/questions/29586520/can-one-get-hierarchical-graphs-from-networkx-with-python-3/29597209#29597209?newreg=dcff13a7effb4b1985b768cbcebadc66

Function to define the Main Progenitor

Function to define Sub(sub)-halos

Function to calculate the doubblecount in mass

In [ ]:
"""def hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, 
                  pos = None, parent = None):
    '''If there is a cycle that is reachable from root, then this will see infinite recursion.
       G: the graph
       root: the root node of current branch
       width: horizontal space allocated for this branch - avoids overlap with other branches
       vert_gap: gap between levels of hierarchy
       vert_loc: vertical location of root
       xcenter: horizontal location of root
       pos: a dict saying where all nodes go if they have been assigned
       parent: parent of this branch.'''
    if pos == None:
        pos = {root:(xcenter,vert_loc)}
    else:
        pos[root] = (xcenter, vert_loc)
    neighbors = G.neighbors(root)
    if parent != None:
        neighbors.remove(parent)
    if len(neighbors)!=0:
        dx = width/len(neighbors) 
        nextx = xcenter - width/2 - dx/2
        for neighbor in neighbors:
            nextx += dx
            pos = hierarchy_pos(G,neighbor, width = dx, vert_gap = vert_gap, 
                                vert_loc = vert_loc-vert_gap, xcenter=nextx, pos=pos, 
                                parent = root)
    return pos"""


def hierarchy_pos(G, root, levels=None, width=1., height=1.):
    '''If there is a cycle that is reachable from root, then this will see infinite recursion.
       G: the graph
       root: the root node
       levels: a dictionary
               key: level number (starting from 0)
               value: number of nodes in this level
       width: horizontal space allocated for drawing
       height: vertical space allocated for drawing'''
    TOTAL = "total"
    CURRENT = "current"
    def make_levels(levels, node=root, currentLevel=0, parent=None):
        """Compute the number of nodes for each level
        """
        if not currentLevel in levels:
            levels[currentLevel] = {TOTAL : 0, CURRENT : 0}
        levels[currentLevel][TOTAL] += 1
        neighbors = G.neighbors(node)
        if parent is not None:
            neighbors.remove(parent)
        for neighbor in neighbors:
            levels =  make_levels(levels, neighbor, currentLevel + 1, node)
        return levels

    def make_pos(pos, node=root, currentLevel=0, parent=None, vert_loc=0):
        dx = 1/levels[currentLevel][TOTAL]
        left = dx/2
        pos[node] = ((left + dx*levels[currentLevel][CURRENT])*width, vert_loc)
        levels[currentLevel][CURRENT] += 1
        neighbors = G.neighbors(node)
        if parent is not None:
            neighbors.remove(parent)
        for neighbor in neighbors:
            pos = make_pos(pos, neighbor, currentLevel + 1, node, vert_loc-vert_gap)
        return pos
    if levels is None:
        levels = make_levels({})
    else:
        levels = {l:{TOTAL: levels[l], CURRENT:0} for l in levels}
    vert_gap = height / (max([l for l in levels])+1)
    return make_pos({})

def DefineMPB(G, root, OnBranch = True, Parent = None, MPB = []):
    neighbors = G.neighbors(root)
    if Parent != None:
        neighbors.remove(Parent)
    if len(neighbors)!=0:
        for i in neighbors:
            if ((MergerTree.node[i]['mmp'] == 1) and OnBranch):
                nx.set_node_attributes(G, 'MPB', {i: 1})
                nx.set_node_attributes(G, 'color', {i: 'blue'})
                MPB.append(i)
                MPB = DefineMPB(G, i, Parent = root, MPB = MPB)
            else:
                nx.set_node_attributes(G, 'MPB', {i: 0})
                nx.set_node_attributes(G, 'color', {i: 'red'})
                DefineMPB(G, i, OnBranch = False, Parent = root)
    return MPB
                
def DefineSubHalos(G, root, Parent = None, OnBranch = True, SubSub = False):
    neighbors = G.neighbors(root)
    if Parent != None:
        neighbors.remove(Parent)
    if len(neighbors)!=0:
        for i in neighbors:
            if ((MergerTree.node[i]['upid'] != -1) and OnBranch):
                nx.set_node_attributes(G, 'SubHalo', {i: 1})
                if MergerTree.node[i]['upid'] != MergerTree.node[i]['pid'] and SubSub:
                    nx.set_node_attributes(G, 'color', {i: 'green'})
                else:
                    nx.set_node_attributes(G, 'color', {i: 'yellow'})
                DefineSubHalos(G, i, Parent = root, OnBranch = True)
            elif (MergerTree.node[i]['MPB'] == 1):
                nx.set_node_attributes(G, 'SubHalo', {i: 0})
                DefineSubHalos(G, i, Parent = root, OnBranch = True)
            else:                
                DefineSubHalos(G, i, Parent = root, OnBranch = False)
                nx.set_node_attributes(G, 'SubHalo', {i: 0})
                
def CountDCM(G, root, Parent = None, OnBranch = True, MassAtMerge = None, Doubble_Count = [], MaxDepth = None, Depth = None):
    
    if Depth != None:
        Depth += 1
    
    if (MaxDepth == None) or (MaxDepth >= Depth):
        #Finds the adjcent nodes and removes the parent
        neighbors = G.neighbors(root)
        if Parent != None:
            neighbors.remove(Parent)


        if len(neighbors)!=0:
            for i in neighbors:
                if ((MergerTree.node[i]['SubHalo'] == 1) and OnBranch):

                    if len(neighbors) == 1:
                        if MassAtMerge == None:
                            Doubble_Count = CountDCM(G, i, Parent = root, MassAtMerge = MergerTree.node[i]['M200c'], Doubble_Count = Doubble_Count, MaxDepth = MaxDepth, Depth = Depth)
                        else:
                            Doubble_Count = CountDCM(G, i, Parent = root, MassAtMerge = MassAtMerge, Doubble_Count = Doubble_Count, MaxDepth = MaxDepth, Depth = Depth)
                    else:
                        if MassAtMerge == None:
                            Doubble_Count = CountDCM(G, i, Parent = root, MassAtMerge = MergerTree.node[i]['M200c'], Doubble_Count = Doubble_Count, MaxDepth = MaxDepth, Depth = Depth)
                        else:
                            if MergerTree.node[i]['M200c'] > MassAtMerge:
                                Doubble_Count.append(np.log10(np.power(10, MergerTree.node[i]['M200c']) - np.power(10, MassAtMerge)))
                            Doubble_Count = CountDCM(G, i, Parent = root, Doubble_Count = Doubble_Count, MaxDepth = MaxDepth, Depth = Depth)

                elif((MergerTree.node[i]['SubHalo'] == 0) and OnBranch and MassAtMerge != None):
                    if MergerTree.node[i]['M200c'] > MassAtMerge:
                        Doubble_Count.append(np.log10(np.power(10, MergerTree.node[i]['M200c']) - np.power(10, MassAtMerge)))            
                else:
                    Doubble_Count = CountDCM(G, i, Parent = root, Doubble_Count = Doubble_Count, MaxDepth = MaxDepth, Depth = Depth)

        elif(MassAtMerge != None):
            if MergerTree.node[root]['M200c'] > MassAtMerge:
                Doubble_Count.append(np.log10(np.power(10, MergerTree.node[root]['M200c']) - np.power(10, MassAtMerge)))
    elif(MassAtMerge != None):
        if MergerTree.node[root]['M200c'] > MassAtMerge:
            Doubble_Count.append(np.log10(np.power(10, MergerTree.node[root]['M200c']) - np.power(10, MassAtMerge)))
    return Doubble_Count


def RemoveDCM(G, root, MPB, Depth, Parent = None, OnBranch = True, MassAtMerge = None, Doubble_Count = []):
    
    #Finds the adjcent nodes and removes the parent
    neighbors = G.neighbors(root)
    if Parent != None:
        neighbors.remove(Parent)


    if len(neighbors)!=0:
        for i in neighbors:
            if ((MergerTree.node[i]['SubHalo'] == 1) and OnBranch):

                if len(neighbors) == 1:
                    if MassAtMerge == None:
                        Doubble_Count = RemoveDCM(G, i, MPB, Depth+1, Parent = root, MassAtMerge = MergerTree.node[i]['M200c'], Doubble_Count = Doubble_Count)
                    else:
                        Doubble_Count = RemoveDCM(G, i, MPB, Depth+1, Parent = root, MassAtMerge = MassAtMerge, Doubble_Count = Doubble_Count)
                else:
                    if MassAtMerge == None:
                        Doubble_Count = RemoveDCM(G, i, MPB, Depth+1, Parent = root, MassAtMerge = MergerTree.node[i]['M200c'], Doubble_Count = Doubble_Count)
                    else:
                        if MergerTree.node[i]['M200c'] > MassAtMerge:
                            Doubble_Count.append(np.log10(np.power(10, MergerTree.node[i]['M200c']) - np.power(10, MassAtMerge)))
                        Doubble_Count = RemoveDCM(G, i, MPB, Depth+1, Parent = root, Doubble_Count = Doubble_Count)

            elif((MergerTree.node[i]['SubHalo'] == 0) and OnBranch and MassAtMerge != None):
                if MergerTree.node[i]['M200c'] > MassAtMerge:
                    Doubble_Count.append(np.log10(np.power(10, MergerTree.node[i]['M200c']) - np.power(10, MassAtMerge)))            
            else:
                Doubble_Count = RemoveDCM(G, i, MPB, Depth+1, Parent = root, Doubble_Count = Doubble_Count)

    elif(MassAtMerge != None):
        if MergerTree.node[root]['M200c'] > MassAtMerge:
            Doubble_Count.append(np.log10(np.power(10, MergerTree.node[root]['M200c']) - np.power(10, MassAtMerge)))
    
    if len(neighbors) > 1:
        Doubble_Count = []
    else:
        MergerTree.node[MPB[Depth]]['M200c'] = np.log10(np.power(10, MergerTree.node[MPB[Depth]]['M200c']) - np.sum(np.power(10, Doubble_Count)))
    return Doubble_Count

Reads DF to create network 

In [ ]:
DF = df[df.scale > 0.25]
MergerTree = nx.from_pandas_dataframe(DF.drop(0), "id", "desc_id", edge_attr = "mmp", create_using = nx.Graph())
nx.set_node_attributes(MergerTree, 'scale', pd.Series(DF.scale.values, index=DF.id).to_dict())
nx.set_node_attributes(MergerTree, 'M200c', pd.Series(DF.M200c.values, index=DF.id).to_dict())
nx.set_node_attributes(MergerTree, 'mmp', pd.Series(DF.mmp.values, index=DF.id).to_dict())
nx.set_node_attributes(MergerTree, 'upid', pd.Series(DF.upid.values, index=DF.id).to_dict())
nx.set_node_attributes(MergerTree, 'pid', pd.Series(DF.pid.values, index=DF.id).to_dict())

In [ ]:
#Sets Attributes of root
nx.set_node_attributes(MergerTree, 'color', {2812608373: 'blue'})

#sets the colors and MP/SUB properties of the halos
MPB = DefineMPB(MergerTree, 2812608373, MPB = [2812608373])
DefineSubHalos(MergerTree, 2812608373)

#makes color mapping for plotting hte nodes
nodes = MergerTree.nodes()
colors = [MergerTree.node[n]['color'] for n in nodes]

#Makes the positioning of the nodes
pos = hierarchy_pos(MergerTree, 2812608373) #2nd input is the root (MP at z = 0)

#'draws' the graph
nx.draw(MergerTree, pos = pos, nodelist = nodes, node_color = colors, with_labels = False, node_size = 10)

#calculates the doubble count of mass
Out = CountDCM(MergerTree, 2812608373, Doubble_Count= [])
print(Out)
print(np.log10(np.sum(np.power(10, np.array(Out)))))

Morig = [MergerTree.node[n]['M200c'] for n in MPB]
RemoveDCM(MergerTree, 2812608373, MPB, 0, Doubble_Count= [])
Mcorr = [MergerTree.node[n]['M200c'] for n in MPB]
Mdiff = np.log10(np.power(10, Morig) - np.power(10, Mcorr))
print(np.column_stack((Morig,Mcorr,Mdiff)))

In [ ]:
def MassSubtraction(G, root, Parent = None):
    MPB  = DefineMPB(G, root, Parent = None, MPB = []) #gets the main projenitor branch
    MaxDepth = len(MPB)
    Outs = []
    for i in range(0, MaxDepth):
        Out = CountDCM(MergerTree, 2812608373, Doubble_Count = [], MaxDepth = i, Depth = 0)
        Outs.append(np.log10(np.sum(np.power(10, np.array(Out)))))
    
    Morig = [MergerTree.node[n]['M200c'] for n in MPB]
    
    for i in range(0, MaxDepth - 1):
        MergerTree.node[MPB[i]]['M200c'] = np.log10(np.power(10, MergerTree.node[MPB[i]]['M200c']) - (np.power(10, Outs[i+1]) -  np.power(10, Outs[i])))
    
    Mcorr = [MergerTree.node[n]['M200c'] for n in MPB]
    
    return Morig, Mcorr

In [ ]:
MassSubtraction(MergerTree, 2812608373)

In [ ]:
A = np.array([12.110825310054965, 12.110421346473956, 12.113642582771741, 12.114377429786156, 12.11169927757355, 12.110993517377995, 12.104589527179373, 12.100542850012465, 12.099197055379921, 12.096562438374136, 12.091596620810058, 12.08700012079599, 12.084111351940678, 12.082928915015131, 12.077803798076088, 12.074304344001435, 12.071513805095089, 12.069520100835227, 12.066214075471244, 12.061150780928548, 12.059298292408071, 12.055416559840488, 12.050534419540124, 12.04879127384841, 12.046300019652969, 12.044422155711842, 12.048130927028968, 12.043558746914734, 12.048053173115608, 12.046924174230035, 12.045870392449361, 12.044657333234866, 12.043440876244473, 12.042102768037303, 12.039414119176136, 12.039889797736182, 12.036149629745854, 12.03726709456871, 12.042654253167793, 12.043440876244473, 12.040760524228698, 12.038977622332691, 12.037147503632555, 12.03973129609869, 12.04123473117143, 12.042654253167793, 12.045322978786658, 12.046182890740882, 12.048635988432649, 12.050341081834084, 12.047080072816257, 12.044108387446119, 12.040760524228698, 12.038659894302496, 12.035469763481283, 12.031044716583706, 12.026819159241226, 12.023005397249936])
B = np.array([12.110825310054965, 12.110421346473956, 12.113642582771741, 12.114377429786156, 12.11169927757355, 12.110993517377995, 12.10379534275325, 12.100329250710812, 12.099197055379921, 12.096562438374136, 12.091596620810058, 12.086945021831385, 12.084111351940678, 12.082873297074634, 12.077803798076088, 12.074304344001435, 12.071513805095089, 12.069520100835227, 12.066214075471244, 12.060975360251065, 12.059239564155225, 12.055238807154925, 12.050654245708175, 12.048911543063788, 12.046300019652969, 12.044361420025885, 12.048130927028968, 12.043315151565327, 12.048053173115608, 12.046803353314489, 12.0452644037701, 12.044657333234866, 12.043075352388925, 12.041858354430083, 12.039291190492447, 12.039090915576034, 12.036273450756884, 12.036834577692879, 12.041860432860707, 12.043258152772083, 12.03971762262224, 12.038731443102604, 12.037023931606175, 12.039547044503246, 12.041173469053268, 12.042471198364751, 12.04495899707886, 12.045698461586236, 12.048696129297596, 12.050101262181155, 12.04671791236709, 12.043621676752311, 12.040637936563574, 12.037797044025428, 12.035221627249944, 12.028720221052769, 12.024471710941462, 12.023005397249936])


C = np.log10(np.power(10, A) - np.power(10, B))

print(C)
C = np.nan_to_num(C)
C[C == 0] = -np.inf
print(C)
print(np.log10(np.sum(np.power(10, C))))

In [ ]:
CountDCM(MergerTree, 2812608373, Doubble_Count = [], MaxDepth = 10, Depth = 0)

In [ ]:
MPB  = DefineMPB(MergerTree, 2812608373, Parent = None, MPB = []) #gets the main projenitor branch
MaxDepth = len(MPB)
print(MaxDepth)